# Understanding IfcRevolvedAreaSolid



In [1]:
import ada
from ada.core.curve_utils import get_center_from_3_points_and_radius
from ada.core.vector_utils import unit_vector, angle_between, normal_to_points_in_plane, transform3d, EquationOfPlane
from ada.ifc.utils import to_real, create_ifc_placement
from ada.core.constants import X, Y ,Z, O
import ifcopenshell
import numpy as np

In [2]:
def pipe_w_multiple_bends() -> ada.Pipe:
    z = 3.2
    y0 = -200e-3
    x0 = -y0
    coords = [
        (0, y0, z),
        (5 + x0, y0, z),
        (5 + x0, y0 + 5, z),
        (10, y0 + 5, z + 2),
        (10, y0 + 5, z + 10),
    ]
    pipe1 = ada.Pipe(
        "Pipe1",
        coords,
        ada.Section("PSec", "PIPE", r=0.10, wt=5e-3),
    )

    return ada.Assembly('TestModel') / ada.Part('TestSite') /  pipe1

a = pipe_w_multiple_bends()
pipe = list(a.get_all_physical_objects())[0]
pipe_bends = list(filter(lambda x: isinstance(x, ada.PipeSegElbow), pipe.segments))

in_plane_pipe_bend = pipe_bends[0]
out_of_plane_pipe_bend = pipe_bends[1]

## In-plane example

In [3]:
pipe_elbow = in_plane_pipe_bend

xvec1 = unit_vector(pipe_elbow.xvec1)
xvec2 = unit_vector(pipe_elbow.xvec2)

p1, p2, p3 = pipe_elbow.p1.p, pipe_elbow.p2.p, pipe_elbow.p3.p

normal = normal_to_points_in_plane([p1, p2, p3])

revolve_angle = np.rad2deg(angle_between(xvec1, xvec2))
cd = get_center_from_3_points_and_radius(p1, p2, p3, pipe_elbow.bend_radius)
arc_p1 = pipe_elbow.arc_seg.p1
diff = cd.center - arc_p1

yvec = np.cross(xvec1, normal)
global_csys = (X, Y, Z)
new_csys = (normal, yvec, xvec1)

n_tra, diff_tra = transform3d(global_csys, new_csys, O, [normal, diff])

n_tra_norm = to_real(unit_vector(n_tra))
diff_tra_norm = to_real(diff_tra)

# Calculate back the transformed coordinate diff_tra_norm (it is represented in a local coordinate system "new_csys")
abs_coord = transform3d(new_csys, global_csys, O, [diff])[0] + cd.center

# Is the axis point in the XY plane
eqp = EquationOfPlane(arc_p1, normal)
eqp.is_point_in_plane(abs_coord)

True

In [4]:
diff.dot(eqp.normal)

0.0

## Out-of-plane Example

In [10]:
pipe_elbow = out_of_plane_pipe_bend

xvec1 = unit_vector(pipe_elbow.xvec1)
xvec2 = unit_vector(pipe_elbow.xvec2)
p1, p2, p3 = pipe_elbow.p1.p, pipe_elbow.p2.p, pipe_elbow.p3.p

normal = normal_to_points_in_plane([p1, p2, p3])

revolve_angle = np.rad2deg(angle_between(xvec1, xvec2))
cd = get_center_from_3_points_and_radius(p1, p2, p3, pipe_elbow.bend_radius)
arc_p1 = pipe_elbow.arc_seg.p1
diff = cd.center - arc_p1

yvec = np.cross(xvec1, normal)
global_csys = (X, Y, Z)
new_csys = (normal, yvec, xvec1)

n_tra, diff_tra = transform3d(global_csys, new_csys, O, [normal, diff])

n_tra_norm = to_real(unit_vector(n_tra))
diff_tra_norm = to_real(diff_tra)

# Calculate back the transformed coordinate diff_tra_norm (it is represented in a local coordinate system "new_csys")
abs_coord = transform3d(new_csys, global_csys, O, [diff])[0] + cd.center

eqp = EquationOfPlane(arc_p1, normal)
eqp.is_point_in_plane(abs_coord)

False

Okay, so the point is not in the XY-plane. How about projecting the point back into the XY-plane

In [11]:
dist = diff.dot(eqp.normal)
proj_diff = diff - dist*eqp.normal

dist, diff, proj_diff

(2.0816681711721685e-16,
 array([0.18088442, 0.        , 0.07536851]),
 array([0.18088442, 0.        , 0.07536851]))